In [2]:
import keras
import os
import json





In [3]:
"""phase 1- delft 15"""

data_dir='../code/data_dkt/delft15_phase1/'
skill_dict=json.load(open('data_dkt/delft15_phase2/skill_dict_delft_all_0_336_all.json'))
skill=os.path.join(data_dir+"skill_df_delft_15_phase1.csv")
response=os.path.join(data_dir+"response_df_delft_15_phase1.csv")




"""phase 1- delft 14"""


# data_dir='../code/data_dkt/delft14_phase1/'
# skill_dict=json.load(open('data_dkt/delft15_phase2/skill_dict_delft_all_0_336_all.json'))
# skill=os.path.join(data_dir+"skill_df_delft_14_phase1.csv")
# response=os.path.join(data_dir+"response_df_delft_14_phase1.csv")

'phase 1- delft 14'

In [4]:

"""phase 2 """
# data_dir = "Delft15_phase2/"
# skill_dict=json.load(open(os.path.join(data_dir,"unique_events.json")))
# response = os.path.join(data_dir,"effort_response.csv")
# skill = os.path.join(data_dir,"skill.csv")


'phase 2 '

In [5]:

!ls ../code/data_dkt/delft15/

ls: cannot access ../code/data_dkt/delft15/: No such file or directory


# data_helper.py

In [6]:
import pandas as pd
import numpy as np
import json
import os


def load_data(skill_csv,response_csv,skill_json):
    """
    This function reads data files and returns:
    1. skill_dict:reads data from __skill_dict.json__ which maps skills to numbers 1,2..
    2. skill_df: reads data from __skill.tsv__ which is a sequence of exercise or skills attempted by a student
    3. response_df: reads data from correct.tsv which is sequence of binary response to skill/exercise in skill.tsv by each student
    4. assistment_df: reads data from assistment_id.tsv which is a sequence of event id of the skill/exercises

    """
    #response_df = pd.read_csv('correct.tsv', sep='\t').drop('Unnamed: 0', axis=1)
    # skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
    # assistment_df = pd.read_csv('assistment_id.tsv', sep='\t').drop('Unnamed: 0', axis=1)
    # data_dir="data/dktData/"
    data_dir=""
    response_csv = os.path.join(data_dir,response_csv)

    print(response_csv)

    # response_df = pd.read_csv(response_csv, sep='\t').drop('Unnamed: 0', axis=1)
    response_df = pd.read_csv(response_csv)
    print(1)
    skill_csv = os.path.join(data_dir,skill_csv)
    # skill_df=pd.read_csv(skill_csv, sep='\t').drop('Unnamed: 0', axis=1)
    skill_df=pd.read_csv(skill_csv)
#     print(2)
#     skill_dict=json.load(open(skill_json))
#     print(3)

    print(response_df.shape,skill_df.shape)
    return response_df, skill_df, skill_dict


def preprocess(skill_df, response_df, skill_num):
    """
    This function extracts the skills and responses from the loaded files excluding student ids
    :param skill_df: skills attempted by a student at each timestep
    :param response_df: responses on the exercises  by a student at each timestep
    :param skill_num: Total number of skills in the course
    """
    skill_matrix = skill_df.iloc[:, 1:].values - 1 # 0 indexing skill numbers
    response_array = response_df.iloc[:, 1:].values
    skill_array = one_hot(skill_matrix, skill_num)
    skill_response_array = dkt_one_hot(skill_matrix, response_array, skill_num)
    return skill_array, response_array, skill_response_array

def one_hot(skill_matrix, vocab_size):
    """
    params:
        skill_matrix: 2-D matrix (student, skills)
        vocal_size: Number of skills in the course
    returns:
        a 3d-darray with a shape like (student, sequence_len, vocab_size)
    """

    seq_len = skill_matrix.shape[1] # sequence length is the number of the skills
    # instantiation of a numpy array with all elements 0
    result = np.zeros((skill_matrix.shape[0], seq_len, vocab_size))
    # iterate over each student in data
    for i in range(skill_matrix.shape[0]):
        # set vocabulary values in skills attempted by a student equal to 1
        result[i, np.arange(seq_len), skill_matrix[i]] = 1.
    return result

def dkt_one_hot(skill_matrix, response_matrix, vocab_size):
    """
   params:
       skill_matrix: 2-D matrix (student, skills)
       response_matrix:  2-D matrix (student, responses)
       vocal_size: Number of skills in the course
   returns:
       a 3d-darray with a shape like (student, sequence_len, 2*vocab_size)
   """

    seq_len = skill_matrix.shape[1]# sequence length is the number of the skills
    # instantiation of a numpy array with all elements 0
    skill_response_array = np.zeros((skill_matrix.shape[0], seq_len, 2 * vocab_size))
    # iterate over each student in data
    for i in range(skill_matrix.shape[0]):
        # set vocabulary values in correct response attempted by a student equal to 1
        # 2* skill_matrix[i] goes to index in one hot encode for responses

        skill_response_array[i, np.arange(seq_len), 2 * skill_matrix[i] + response_matrix[i]] = 1.
    return skill_response_array



# DKT.PY

In [7]:
from keras.models import Model
from keras.layers import Input, Dropout, Masking, Dense, Embedding
from keras.layers import Embedding
from keras.layers.core import Flatten, Reshape
from keras.layers import LSTM
from keras.layers import merge
from keras.layers.merge import multiply
from keras.callbacks import EarlyStopping
from keras import backend as K
from theano import tensor as T
from theano import config
from theano import printing
from theano import function
from keras.layers import Lambda
import theano
import numpy as np
import pdb
from math import sqrt
from keras.callbacks import Callback



class DKTnet():

    def __init__(self, 
                input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=[], 
                y_train=[], 
                y_train_order=[],
                validation_split=0.0,
                validation_data=None,
                optimizer='adam',
                callbacks=None):
        """

        :param input_dim: dimension of the input at one timestamp (dimension of x_t= 2*num_skills)
        :param input_dim_order: dimension of the one-hot representation of problem to check order of occurence(=num_skills)
        :param hidden_layer_size: number of nodes in hidden layer
        :param x_train: 3D matrix of size (samples, number of timestamp/sequence length, dimension of input vec (x_t) )
        :param y_train: a matrix of responses (samples,number of timestamp/sequence length)
        :param y_train_order: shape of output equal to number of timesteps
        :param validation_split:
        :param validation_data:
        :param optimizer:
        :param callbacks:

        """
        ## input dim is the dimension of the input at one timestamp (dimension of x_t)
        self.input_dim = int(input_dim) #2* num_skills

        ## input_dim_order is the dimension of the one-hot representation of problem
        ## CHECK: order of occurence of responses should be according to timestamp
        self.input_dim_order = int(input_dim_order)#num_skills

        self.hidden_layer_size = int(hidden_layer_size)
        self.batch_size = int(batch_size)
        self.epochs = int(epochs)

        ## x_train is a 3D matrix of size (samples, number of timestamp, dimension of input vec (x_t) )
        ## in cognitive tutor # of students * # total responses * # input_dim
        self.x_train = x_train
        ## y_train is a matrix of (samples one hot representation according to problem output value at each timestamp (y_t) )
        self.y_train = y_train
        ## y_train_order is the one hot representation of problem according to timestamp starting from
        ## t=1 if training starts at t=0
        self.y_train_order = y_train_order
        # users: no of student datapoints
        self.users = np.shape(x_train)[0]
        self.validation_split = validation_split
        self.validation_data = validation_data
        self.optimizer = optimizer
        self.callbacks = callbacks
        print ("DKTnet initialization done")

    def build_train_on_batch(self):
        ## 1. First layer for the input (x_t), creates a tensor object
        x = Input(batch_shape=(None, None, self.input_dim), name='x')

        ## 2. Mask unknown or anomalous valued timesteps in x
        # the timestep will be masked (skipped) if all values in the input tensor
        #  at that timestep are equal to mask_value
        masked = Masking(mask_value=-1)(x)

        ## 3. Add a lstm layer, return sequences is True to allow output have same
        # dimension as number of timesteps in input
        lstm_out = LSTM(self.hidden_layer_size, return_sequences=True)(masked)

        ## 4. Add a fully connected layer on lstm layer
        dense_out = Dense(self.input_dim_order, activation='sigmoid')(lstm_out)
        ## 5. Create  a tensor object --not sure if its required
        y_order = Input(batch_shape=(None, None, self.input_dim_order), name='y_order')
        merged = multiply([dense_out, y_order])

        def reduce_dim(x):
            #Custom tensor arithmatic from backend K
            # chooses the max value from the output of previous layer
            # this will reduce dimension from skill_num to 1 for each timestep
            x = K.max(x, axis=2, keepdims=True)
            return x

        def reduce_dim_shape(input_shape):
            shape = list(input_shape)
            shape[-1] = 1
            return tuple(shape)

        ## 6. Chooses the max value from the output of previous layer
            # this will reduce dimension from skill_num to 1 for each timestep
        reduced = Lambda(reduce_dim, output_shape=reduce_dim_shape)(merged)

        ## 7. Creates model object with specified input and output
        self.model = Model(inputs=[x, y_order], outputs=reduced)

        ## 8. Compile model by assigning loss function for backpropagtion
        self.model.compile(optimizer=self.optimizer,
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        print('Summary of the model')
        self.model.summary()

    def train_on_batch(self, x_train, y_train, y_train_order):

        self.model.train_on_batch([x_train, y_train_order], y_train)



    def test_on_batch(self, x_val, y_val, y_val_order):
        """
       Test the model on a single batch of samples.
       :param x_train:
       :param y_train:
       :param y_train_order:
       :return: Scalar test loss (if the model has a single output and no metrics) or list of scalars (if the model has multiple outputs and/or metrics).
       The attribute model.metrics_names will give you the display labels for the scalar outputs.
       """
        print(self.model.metrics_names)
        return self.model.test_on_batch([x_val, y_val_order], y_val)

    def predict(self, x_val, y_val_order):
        return self.model.predict([x_val, y_val_order])

    

    def fit_data(self):
        self.model.fit([self.x_train, self.y_train_order], 
                self.y_train, 
                batch_size=self.batch_size,
                epochs=self.epochs,
                callbacks=self.callbacks,
                validation_split=self.validation_split, 
                validation_data=self.validation_data,
                shuffle=True)


# train.py

In [ ]:
import pandas as pd
import numpy as np
import json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import argparse

# from DKT import DKTnet
# from data_helper import load_data, one_hot, preprocess

def get_callbacks():
    '''
    Some callback functions that you may find useful.
    Please refer to https://keras.io/callbacks/ for more detailed explaination
    '''
    checkpoint = ModelCheckpoint('my_model', 
                                 monitor='val_loss',
                                 verbose=2, 
                                 save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=2)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=0, min_lr=1e-4)
    return [checkpoint, early_stopping, reduce_lr]

def train(skill_array, response_array, skill_response_array):
    '''
    train the model with 
    '''
    input_dim = skill_response_array.shape[-1] #2* num_skills
    input_dim_order = skill_array.shape[-1] #num_skills
    hidden_layer_size = 40
    batch_size = 32
    epochs = 5

    dkt = DKTnet(input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=skill_response_array[:, :-1, :], 
                y_train=response_array[:, :-1, np.newaxis], 
                y_train_order=skill_array[:, 1:, :],
                validation_split=0.1,
                validation_data=None,
                optimizer='adam',
                callbacks=get_callbacks())
    dkt.build()
    dkt.fit_data()


def batch_generator(skill_array, response_array, skill_response_array, batch_size=64, shuffle=True):
    """
    return: batches of data from the original data set for training
    """
    sample_num = skill_array.shape[0]
    if shuffle:
        shuffled_indices = np.random.permutation(sample_num)
        skill_array = skill_array.copy()[shuffled_indices]
        response_array = response_array.copy()[shuffled_indices]
        skill_response_array = skill_response_array.copy()[shuffled_indices]
        print('Training set shuffled')
    for ndx in range(0, sample_num, batch_size):
        skill_array_batch = skill_array[ndx:min(ndx+batch_size, sample_num)]
        response_array_batch = response_array[ndx:min(ndx+batch_size, sample_num)]
        skill_response_array_batch = skill_response_array[ndx:min(ndx+batch_size, sample_num)]
        yield skill_response_array_batch, response_array_batch, skill_array_batch

def create_validation_data(skill_array, response_array, skill_response_array,size=0.2):
    """
    return: split of data from the original data set for testing
    """
    sample_num = skill_array.shape[0]
    shuffled_indices = np.random.permutation(sample_num)
    skill_array = skill_array.copy()[shuffled_indices]
    response_array = response_array.copy()[shuffled_indices]
    skill_response_array = skill_response_array.copy()[shuffled_indices]
    print('Data set shuffled, preparing for split')
    split_index=int(size*sample_num)
    train_index=sample_num-split_index
    print('train:', train_index, "test:",split_index)
    skill_array_train = skill_array[0:train_index]
    response_array_train = response_array[0:train_index]
    skill_response_array_train = skill_response_array[0:train_index]
    
    skill_array_test = skill_array[train_index:]
    response_array_test = response_array[train_index:]
    skill_response_array_test = skill_response_array[train_index:]
    
    return skill_array_train, response_array_train, skill_response_array_train,skill_array_test,response_array_test,skill_response_array_test


def train_on_batch(skill_array, response_array, skill_response_array):
    """This function creates a DKT MODEL object using DKT.py and
    trains it using the batched data"""

    input_dim = skill_response_array.shape[-1]  #2* num_skills
    input_dim_order = skill_array.shape[-1] #num_skills
    hidden_layer_size = 40
    batch_size = 64
    epochs = 5
    print("batch size=",batch_size)
    print("epoch size=", epochs)
    

    '''
    parameters like batch_size, epochs x_train, y_train, y_train_order and validations are useless
    if you are doing a training by batch
    '''
    dkt = DKTnet(input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=skill_response_array[:, :-1, :], 
                y_train=response_array[:, :-1, np.newaxis], 
                y_train_order=skill_array[:, 1:, :],
                validation_split=0.2,
                validation_data=0.2,
                optimizer='adam',
                callbacks=None)

    dkt.build_train_on_batch()

    '''
    For simplification, we are over fitting on the training set here.
    In your model, you should do a train-test split or cross-validation which can be found in sklearn package.
    '''
    skill_array_train, response_array_train, skill_response_array_train,skill_array_test,response_array_test,skill_response_array_test=create_validation_data(skill_array, response_array, skill_response_array,size=dkt.validation_split)




    for e in range(epochs):
        print('***Epoch', e+1, 'starts****')
        iteration = 0
        total_iteration_num = 1 + (skill_array.shape[0] - 1) // batch_size
        for skill_response_array_batch, response_array_batch, skill_array_batch in batch_generator(skill_array_train, response_array_train, skill_response_array_train, batch_size=batch_size):
            dkt.train_on_batch(skill_response_array_batch[:, :-1, :],response_array_batch[:, :-1, np.newaxis],  skill_array_batch[:, 1:, :])
            iteration += 1
        print("iter: {}/{} done".format(iteration, total_iteration_num))
        
        result = dkt.test_on_batch(skill_response_array_test[:, :-1, :],response_array_test[:, :-1, np.newaxis],  skill_array_test[:, 1:, :])
        trainresult = dkt.test_on_batch(skill_response_array_train[:, :-1, :],response_array_train[:, :-1, np.newaxis],  skill_array_train[:, 1:, :])
        print('Evalutaion training data result', trainresult)
        print('Evalutaion validation data result', result)
        '''
        You should implement your own evaluation function here to evaluate your result on the validation set if each sample have different timesteps
        '''
    prediction = dkt.predict(skill_response_array[0:1, :-1, :],
                                skill_array[0:1, 1:, :])

    print('Check Prediction Output:', prediction,response_array[:, :-1, np.newaxis])
    # print('Check shape:Input,prediction, y_train:',dkt.x_train.shape, prediction.shape,dkt.y_train[0:1].shape)
    
    
    
    
    
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-S',"--skill_df", type=str, required=True)
#     parser.add_argument('-R',"--response_df",type=str, required=True)
#     parser.add_argument('-dict',"--skill_dict", type=str, required=True)
#     args = parser.parse_args()
#     print(args)

#     response_df,skill_df,skill_dict=load_data(args.skill_df,args.response_df,args.skill_dict)
#     # response_df,skill_df,skill_dict=load_data()
#     skills_num = len(skill_dict)
#     print('Number of skills are :{}'.format(skills_num))
#     skill_array, response_array, skill_response_array = preprocess(skill_df, response_df, skills_num)
#     # train(skill_array, response_array, skill_response_array)
#     train_on_batch(skill_array, response_array, skill_response_array)

#     # python train.py -S skill_df_delft_15.csv -R response_df_delft_15.csv -dict skill_dict_delft_15.json
    

In [ ]:
response_df,skill_df,skill_dict=load_data(skill,response,skill_dict)
skills_num = len(skill_dict)
print('Number of skills are :{}'.format(skills_num))
skill_array, response_array, skill_response_array = preprocess(skill_df, response_df, skills_num)
print("skill_array, response_array, skill_response_array/n",
      skill_array.shape,response_array.shape,skill_response_array.shape)

# print(skill_array[0])
# print(response_array[0])
# print(skill_response_array[0])

# #train(skill_array, response_array, skill_response_array)
train_on_batch(skill_array, response_array, skill_response_array)


../code/data_dkt/delft15_phase1/response_df_delft_15_phase1.csv
1
((9242, 101), (9242, 101))
Number of skills are :336
('skill_array, response_array, skill_response_array/n', (9242, 100, 336), (9242, 100), (9242, 100, 672))
('batch size=', 64)
('epoch size=', 10)
DKTnet initialization done
Summary of the model
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
x (InputLayer)                   (None, None, 672)     0                                            
____________________________________________________________________________________________________
masking_1 (Masking)              (None, None, 672)     0           x[0][0]                          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, None, 40)      114080      masking_1[0][0]